Weight Initialization Tricks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Complete-Deep-Learning/master/ANN/Churn_Modelling.csv')

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
x = df.iloc[:,3:13]
y = df.iloc[:,13]

In [ ]:
geography = pd.get_dummies(x['Geography'],drop_first=True)
gender = pd.get_dummies(x['Gender'],drop_first=True)

In [ ]:
x = pd.concat([x,geography,gender],axis=1)

In [ ]:
x.drop(['Geography','Gender'],axis = 1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
###Feature Scaling is important since computational becomes less, hence faster convergence
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [ ]:
x_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import  Dropout , LeakyReLU,PReLU,ELU

In [ ]:
classifier = Sequential()
classifier.add(Dense(units=6 , kernel_initializer='he_uniform',activation='relu',input_dim=11))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(units=1, kernel_initializer = 'glorot_uniform' ,activation='sigmoid'))
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 6)                 72        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
 classifier.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [ ]:
model_history = classifier.fit(x_train,y_train,validation_split=0.3,batch_size=10,epochs=100)

Epoch 1/100
560/560 [==============================] - 2s 2ms/step - loss: 0.7076 - accuracy: 0.6682 - val_loss: 0.5037 - val_accuracy: 0.7967
Epoch 2/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5251 - accuracy: 0.7877 - val_loss: 0.4711 - val_accuracy: 0.7958
Epoch 3/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4905 - accuracy: 0.7979 - val_loss: 0.4609 - val_accuracy: 0.7958
Epoch 4/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4876 - accuracy: 0.7987 - val_loss: 0.4574 - val_accuracy: 0.7958
Epoch 5/100
560/560 [==============================] - 1s 3ms/step - loss: 0.4846 - accuracy: 0.7952 - val_loss: 0.4539 - val_accuracy: 0.7967
Epoch 6/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4741 - accuracy: 0.7943 - val_loss: 0.4503 - val_accuracy: 0.7967
Epoch 7/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4705 - accuracy: 0.7980 - val_loss: 0.4484 - val_accuracy: 0.7958

In [ ]:
print(model_history.history.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [ ]:
y_pred = classifier.predict(x_test)
y_pred = (y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score
print(confusion_matrix(y_test,y_pred))
print('score : ',accuracy_score(y_test,y_pred))

[[1581   14]
 [ 316   89]]
score :  0.835


###Keras Tuner - Decide Number of Hidden Layers And Neuron in Neural Network

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=dac2645d579d5a000fbe8e996b11fc514914ddb8eed009a34c8e1f1c29e4e7a8
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=4a019e6cb05808ff3712554c82b7884518fb177c12c6fa732eb5cc0095777ca9
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Keras-Tuner/main/Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

###Hyperparamters
 1.How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [ ]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):
    model.add(layers.Dense(units = hp.Int('units_' + str(i),
                                          min_value=32,
                                          max_value=512,
                                          step=32),
                           activation='relu'))
  model.add(layers.Dense(1,activation='linear'))
  model.compile(optimizer = keras.optimizers.Adam(
      hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
      loss = 'mean_absolute_error',
      metrics = ['mean_absolute_error'])
  return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective = 'val_mean_absolute_error',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'sahil',
    project_name = 'AQI'
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 25s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in sahil/AQI
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 15
units_0: 512
units_1: 256
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 16
units_0: 256
units_1: 416
learning_rate: 0.01
units_2: 416
units_3: 64
units_4: 384
units_5: 160
units_6: 480
units_7: 64
units_8: 352
units_9: 96
units_10: 224
units_11: 352
units_12: 160
units_13: 160
units_14: 448
units_15: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 17
units_0: 160
units_1: 96
learning_rate: 0.001
units_2: 512
units_3: 416
units_4: 64
units_5: 96
units_6: 352
units_7: 448
units_8: 320
units_9: 256
units_10: 96
units_11: 448
units_12: 128
units_13: 288
units_14: 96
units_15: 416
units_16: 32
Score: nan
Trial summary
Hyperparameters:

###HyperParameter Tuning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Complete-Deep-Learning/master/ANN/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
x = df.iloc[:,3:13]
y = df.iloc[:,13]

In [4]:
geography = pd.get_dummies(x['Geography'],drop_first=True)
gender = pd.get_dummies(x['Gender'],drop_first=True)

In [5]:
x = pd.concat([x,geography,gender],axis=1)

In [6]:
x.drop(['Geography','Gender'],axis = 1,inplace=True)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [8]:
###Feature Scaling is important since computational becomes less, hence faster convergence
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [47]:
def create_model(layers,activation):
  model = Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=x_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))

  model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model 

In [48]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Flatten,Dropout,LeakyReLU,BatchNormalization
from keras.activations import relu,sigmoid

In [52]:
model = KerasClassifier(build_fn=create_model,verbose=2)

layers = [(20,),(40,20),(45,30,15)]
activations = ['sigmoid','relu']
param_grid = dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
grid = GridSearchCV(estimator=model,param_grid = param_grid,cv=5)

In [53]:
for i,nodes in enumerate(layers):
  print(i,nodes)
  print("\n")

0 (20,)


1 (40, 20)


2 (45, 30, 15)




In [54]:
grid_result = grid.fit(x_train,y_train)

Epoch 1/30
50/50 - 1s - loss: 0.9945 - accuracy: 0.2675
Epoch 2/30
50/50 - 0s - loss: 0.7583 - accuracy: 0.4706
Epoch 3/30
50/50 - 0s - loss: 0.6207 - accuracy: 0.6531
Epoch 4/30
50/50 - 0s - loss: 0.5466 - accuracy: 0.7412
Epoch 5/30
50/50 - 0s - loss: 0.5033 - accuracy: 0.7814
Epoch 6/30
50/50 - 0s - loss: 0.4845 - accuracy: 0.7886
Epoch 7/30
50/50 - 0s - loss: 0.4715 - accuracy: 0.7987
Epoch 8/30
50/50 - 0s - loss: 0.4651 - accuracy: 0.8000
Epoch 9/30
50/50 - 0s - loss: 0.4591 - accuracy: 0.8014
Epoch 10/30
50/50 - 0s - loss: 0.4535 - accuracy: 0.8037
Epoch 11/30
50/50 - 0s - loss: 0.4528 - accuracy: 0.8048
Epoch 12/30
50/50 - 0s - loss: 0.4489 - accuracy: 0.8044
Epoch 13/30
50/50 - 0s - loss: 0.4500 - accuracy: 0.8064
Epoch 14/30
50/50 - 0s - loss: 0.4483 - accuracy: 0.8047
Epoch 15/30
50/50 - 0s - loss: 0.4460 - accuracy: 0.8069
Epoch 16/30
50/50 - 0s - loss: 0.4445 - accuracy: 0.8059
Epoch 17/30
50/50 - 0s - loss: 0.4422 - accuracy: 0.8069
Epoch 18/30
50/50 - 0s - loss: 0.4409 - 

In [55]:
print(grid_result.best_score_,grid_result.best_params_)

0.8581250071525574 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}


In [56]:
y_pred = grid.predict(x_test)
y_pred = (y_pred>0.5)

16/16 - 0s


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [58]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[1564   31]
 [ 233  172]]
0.868


In [59]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [61]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 766197816831512978, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9100725153608461367
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]